In [1]:
%matplotlib inline
%timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.spatial.distance import pdist, squareform
import fastdtw
import umap

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

In [2]:
# load data
mfp = '/home/ngrav/project/wearables/model_zoo'
bst_modelpkl = os.path.join(mfp, '213-itv52_InceptionTime_GA5.pkl')
bst_trainer = os.path.join(mfp, 'trainer_itv52_InceptionTime_GA5.pkl')

pfp = '/home/ngrav/project/wearables/results/'
md = pd.read_csv(os.path.join(pfp, 'md_v52_220111.csv'), index_col=0)

In [3]:
def loadpkl(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
        f.close()
    return data
trainer = loadpkl(bst_trainer)

In [5]:
train_ids = trainer.data.train_ids + trainer.data.val_ids

In [11]:
2084 + 737 - 358

2463

In [12]:
# are they in the train or val set?
len([i for i in trainer.data.test_ids if i in trainer.data.train_ids])

0

In [14]:
# are they in the train or val set?
len([i for i in trainer.data.test_ids if i in trainer.data.val_ids])

358

In [7]:
len(train_ids)

2084

In [8]:
len(trainer.data.test_ids)

737

In [9]:
len([i for i in trainer.data.test_ids if i in train_ids])

358

In [8]:
counter = 0
for j, dataloader in enumerate([trainer.data.train_dl, trainer.data.val_dl, trainer.data.test_dl]):
    dataloader.num_workers = 1
    for i, batch in enumerate(dataloader):
        x, y, idx = batch['x'], batch['y'], batch['id']
        if counter==0:
            idx_total = idx
        else:
            idx_total = idx_total + idx
            
        if len(idx_total) > len(np.unique(idx_total)):
            print('j:{}\ti:{}\tcounter:{}'.format(j, i, counter))
        counter+=1


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


j:2	i:0	counter:66
j:2	i:1	counter:67
j:2	i:2	counter:68
j:2	i:3	counter:69
j:2	i:4	counter:70
j:2	i:5	counter:71
j:2	i:6	counter:72
j:2	i:7	counter:73
j:2	i:8	counter:74
j:2	i:9	counter:75
j:2	i:10	counter:76
j:2	i:11	counter:77
j:2	i:12	counter:78
j:2	i:13	counter:79
j:2	i:14	counter:80
j:2	i:15	counter:81
j:2	i:16	counter:82
j:2	i:17	counter:83
j:2	i:18	counter:84
j:2	i:19	counter:85
j:2	i:20	counter:86
j:2	i:21	counter:87
j:2	i:22	counter:88
j:2	i:23	counter:89


In [6]:
print(len(idx_total))

2821


In [7]:
len(np.unique(idx_total))

2463

In [27]:
# question now is what to do. is it error in the splitter or shuffler?

# if error in the splitter, we would have duplicates in train vs. val or maybe in each set. Try this
counter = 0
for dataloader in [trainer.data.test_dl]:#, trainer.data.val_dl, trainer.data.test_dl]:
    dataloader.num_workers = 1
    for i, batch in enumerate(dataloader):
        x, y, idx = batch['x'], batch['y'], batch['id']
        if counter==0:
            idx_total = idx
        else:
            idx_total = idx_total + idx
        counter+=1
print('len idx minus len unique: ', len(idx_total) - len(np.unique(idx_total)))

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


len idx minus len unique:  0


In [ ]:
# question now is what to do. is it error in the splitter or shuffler?

# if error in the splitter, we would have duplicates in train vs. val or maybe in each set. Try this
counter = 0
for dataloader in [trainer.data.test_dl]:#, trainer.data.val_dl, trainer.data.test_dl]:
    dataloader.num_workers = 1
    for i, batch in enumerate(dataloader):
        x, y, idx = batch['x'], batch['y'], batch['id']
        if counter==0:
            idx_total = idx
        else:
            idx_total = idx_total + idx
        counter+=1
print('len idx minus len unique: ', len(idx_total) - len(np.unique(idx_total)))

# problem source

## overlap in val/test ids

This occurs because after the call for `val_pids` to randomly select 50% of the `test_pids`, there is no 

In [16]:
pids = np.unique([i.split('_')[0] for i in trainer.data['data']['IDs'].keys()])
train_pids = np.random.choice(pids, int(len(pids)*train_ratio*self.prop_trainset), replace=False)
test_pids = [i for i in pids if i not in train_pids]

TypeError: 'torch_dataloaders' object is not subscriptable

In [17]:
val_pids = np.random.choice(test_pids, int(len(test_pids)*0.5), replace=False)

NameError: name 'test_pids' is not defined

In [ ]:
# add to code:
test_pids = [i for i in test_pids if i not in val_pids]

# NOTE:

when creating the metadata, just don't call val dataloader again